In [4]:
!pip install scikit-image

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.6/12.8 MB 19.8 MB/s eta 0:00:01
     ----------- ---------------------------- 3.6/12.8 MB 57.4 MB/s eta 0:00:01
     ------------------- -------------------- 6.3/12.8 MB 50.3 MB/s eta 0:00:01
     ----------------------------------- --- 11.7/12.8 MB 81.8 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 73.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from pathlib import Path
import numpy as np
from skimage import io
from skimage.measure import label, regionprops
from skimage.transform import resize
from tqdm import tqdm

# Function to resize or pad to 640x640
def crop_and_resize(cell_img, size=640):
    h, w = cell_img.shape[:2]
    
    # Compute scale to fit largest dimension to size
    scale = size / max(h, w)
    new_h, new_w = int(h * scale), int(w * scale)
    
    resized = resize(cell_img, (new_h, new_w), preserve_range=True, anti_aliasing=True).astype(cell_img.dtype)
    
    # Pad to size x size
    pad_h = size - new_h
    pad_w = size - new_w
    pad_top = pad_h // 2
    pad_bottom = pad_h - pad_top
    pad_left = pad_w // 2
    pad_right = pad_w - pad_left
    
    if resized.ndim == 2:
        padded = np.pad(resized, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant', constant_values=0)
    else:
        padded = np.pad(resized, ((pad_top, pad_bottom), (pad_left, pad_right), (0,0)), mode='constant', constant_values=0)
    
    return padded

# Paths
mask_root = Path("cellpose_results2")
img_root = Path("data")
output_root = Path("cell_crops")

channels = ["blue", "red", "yellow"]

for ch in channels:
    mask_folder = mask_root / ch / "png"
    img_folder = img_root / ch
    out_folder = output_root / ch
    out_folder.mkdir(parents=True, exist_ok=True)
    
    mask_files = list(mask_folder.glob("*_masks.png"))
    
    for mask_file in tqdm(mask_files, desc=f"Processing {ch}"):
        img_file = img_folder / (mask_file.stem.replace("_masks", "") + ".jpg")
        if not img_file.exists():
            print(f"Image not found for {mask_file.name}")
            continue
        
        img = io.imread(img_file)
        mask = io.imread(mask_file).astype(np.int32)
        
        labeled_mask = label(mask)
        regions = regionprops(labeled_mask)
        
        for i, region in enumerate(regions, 1):
            minr, minc, maxr, maxc = region.bbox
            crop = img[minr:maxr, minc:maxc]
            crop_640 = crop_and_resize(crop, size=640)
            crop_name = f"{mask_file.stem}_cell{i}.png"
            io.imsave(out_folder / crop_name, crop_640)


Processing blue:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\smithm27\Documents\cellpose\cellpose\lib\site-packages\skimage\_shared\utils.py:328: UserWarning: C:\Users\smithm27\Documents\cellpose\cell_crops\blue\B2AI_1_Paclitaxel_A1_R2_z01_blue_masks_cell1.png is a low contrast image
  return func(*args, **kwargs)
c:\Users\smithm27\Documents\cellpose\cellpose\lib\site-packages\skimage\_shared\utils.py:328: UserWarning: C:\Users\smithm27\Documents\cellpose\cell_crops\blue\B2AI_1_Paclitaxel_A1_R2_z01_blue_masks_cell2.png is a low contrast image
  return func(*args, **kwargs)
c:\Users\smithm27\Documents\cellpose\cellpose\lib\site-packages\skimage\_shared\utils.py:328: UserWarning: C:\Users\smithm27\Documents\cellpose\cell_crops\blue\B2AI_1_Paclitaxel_A1_R2_z01_blue_masks_cell3.png is a low contrast image
  return func(*args, **kwargs)
c:\Users\smithm27\Documents\cellpose\cellpose\lib\site-packages\skimage\_shared\utils.py:328: UserWarning: C:\Users\smithm27\Documents\cellpose\cell_crop

In [3]:
import csv

output_root = Path("cell_crops")
csv_save_path = output_root / "pred_cell.csv"
channels = ["red", "yellow", "blue"]

csv_rows = [["r_image", "y_image", "b_image", "g_image", "output_folder", "output_prefix"]]

# Get sorted lists of cropped images per channel
red_files = sorted((output_root / "red").glob("*.png"))
yellow_files = sorted((output_root / "yellow").glob("*.png"))
blue_files = sorted((output_root / "blue").glob("*.png"))

# Find the max number of crops in any channel
max_len = max(len(red_files), len(yellow_files), len(blue_files))

for i in range(max_len):
    r_image = str(red_files[i]) if i < len(red_files) else ""
    y_image = str(yellow_files[i]) if i < len(yellow_files) else ""
    b_image = str(blue_files[i]) if i < len(blue_files) else ""
    g_image = ""
    
    # Use first part of filename as output prefix
    output_prefix = ""
    for f in [r_image, y_image, b_image]:
        if f:
            output_prefix = Path(f).stem.split("_cell")[0] + "_"
            break
    
    csv_rows.append([r_image, y_image, b_image, g_image, str(output_root), output_prefix])

# Write CSV
with open(csv_save_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(csv_rows)

print(f"✅ CSV saved: {csv_save_path}")


✅ CSV saved: cell_crops\pred_cell.csv
